# Router Engine

### Automated Document Summarization and Query System

This code sets up an automated system for summarizing and querying information from a research paper using language models and vector embeddings. The document is loaded, split into manageable chunks, and indexed for both summarization and detailed context retrieval. A router query engine is configured to intelligently select the appropriate tool for answering user queries, leveraging OpenAI's language models for generating responses.








## Setup

In [18]:
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [19]:
# async play nice with python
import nest_asyncio
nest_asyncio.apply()

## Load Data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf


In [20]:
# Loading a document

from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

## Define LLM and Embedding model

In [21]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [22]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## Define Summary Index and Vector Index over the Same Data

In [23]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## Define Query Engines and Set Metadata

In [24]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [25]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

## Define Router Query Engine

In [27]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [28]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: Useful for summarization questions related to MetaGPT.
The document introduces MetaGPT, a meta-programming framework for multi-agent collaboration based on Large Language Models (LLMs). It emphasizes the use of Standardized Operating Procedures (SOPs) to enhance collaboration efficiency and reduce errors among agents. MetaGPT assigns specific roles to agents, streamlining workflows and improving code generation quality. The framework demonstrates state-of-the-art performance in evaluations, showcasing its robustness and efficiency in developing LLM-based multi-agent systems. Additionally, it discusses the importance of structured communication interfaces and executable feedback mechanisms in enhancing code generation quality during runtime. The document also delves into the development process using MetaGPT for software development, outlining the roles of different agents and their contributions throughout the software development lifecycle.


In [29]:
response = query_engine.query("What is the summary of the document in 5 points?")
print(str(response))

Selecting query engine 0: Useful for summarization questions related to MetaGPT.
1. MetaGPT is a meta-programming framework that enhances multi-agent collaboration using human-like Standardized Operating Procedures (SOPs) to streamline workflows and reduce errors.
2. The framework assigns specific roles to agents, breaking down complex tasks into subtasks for efficient collaboration, generating more coherent solutions compared to previous systems.
3. MetaGPT achieves state-of-the-art performance in code generation benchmarks, showcasing its robustness and efficiency in handling complex software projects.
4. It introduces a communication protocol that enhances role communication efficiency and implements structured communication interfaces for agents to exchange information effectively.
5. MetaGPT's design is validated through evaluations on HumanEval and MBPP, demonstrating its potential as a promising meta-programming framework for developing LLM-based multi-agent systems.


In [13]:
print(len(response.source_nodes))

34


In [14]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

Selecting query engine 1: This choice is more relevant as it focuses on retrieving specific context from the MetaGPT paper, which may provide insights into how agents share information with other agents..
Agents share information with other agents by utilizing a shared message pool. This shared message pool allows all agents to exchange messages directly. Agents publish their structured messages in the pool and can also access messages from other entities transparently. This system enables any agent to retrieve required information directly from the shared pool, eliminating the need to inquire about other agents and wait for their responses, thus enhancing communication efficiency.


In [17]:
response = query_engine.query(
    "What is this paper is not about ? "
)
print(str(response))

Selecting query engine 1: This choice is about retrieving specific context from the MetaGPT paper, which is not relevant to determining what the paper is not about..
This paper is not about self-referential self-improvement via prompt evolution, model-agnostic meta-learning for fast adaptation of deep networks, generative model for code infilling and synthesis, speculations concerning the first ultraintelligent machine, chatllm network, learning to learn using gradient descent, or self-planning code generation with large language model.


## Let's put everything together

In [15]:
from utils import get_router_query_engine

query_engine = get_router_query_engine("metagpt.pdf")

In [16]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))

Selecting query engine 1: Ablation study results are specific context from the MetaGPT paper, making choice 2 the most relevant..
The ablation study results provide insights into the impact of removing certain components or features from a system or model. This analysis helps in understanding the importance and contribution of individual elements towards the overall performance or functionality of the system.
